In [ ]:
# default_exp learner

# learner
> API details.

#hide
# module name here

> API details.

In [ ]:
#hide
from nbdev.showdoc import *

In [ ]:
#export
import tensorflow as tf
import tensorflow_addons as tfa
from tensorflow.keras.models import Model
from chitra.datagenerator import Dataset

from functools import partial

import sys, inspect

In [ ]:
#export
def create_classifier(base_model_fn:callable, num_outputs, weights='imagenet', dropout=0.5, name=None):
    base_model = base_model_fn(
        include_top=False,
        input_shape=(None, None, 3),
        weights=weights,
    )
    model = tf.keras.Sequential(name=name)
    model.add(base_model)
    model.add(tf.keras.layers.GlobalAveragePooling2D())
    model.add(tf.keras.layers.Dropout(dropout))
    model.add(tf.keras.layers.Dense(num_outputs, name='output'))
    
    return model

In [ ]:
#export
class Learner(Model):
    _AUTOTUNE = tf.data.experimental.AUTOTUNE
            
    def __init__(self, ds: Dataset, base_model_fn:callable, pretrained=True, **kwargs):
        super(Learner, self).__init__()
        self.ds = ds
        self.total = len(ds)
        self.NUM_CLASSES = ds.NUM_CLASSES
        
        weights = 'imagenet' if pretrained else None
        
        #self.base_model = base_model_fn.name
        
        self.model = create_classifier(
            base_model_fn,
            self.NUM_CLASSES,
            name=kwargs.get('name', None)
        )
        
    
    def build(self): pass
         
    def summary(self): return self.model.summary()

    #def get_layer(name=None, index=None): return self.model(name, index)
    
    def compile(self, *args, **kwargs): return self.model.compile(*args, **kwargs)
    
    def call(self, *args, **kwargs): return self.model.call(*args, **kwargs)
    
    def fit(self, *args, **kwargs): return self.model.fit(*args, **kwargs)
    
    def warmup(self):pass
    
    def rescale(self, image, label):
        image = tf.cast(image, tf.float32)
        image = image / 127.5 - 1.0
        return image, label
    
    def _get_optimizer(self,
                       optimizer,
                       momentum=0.9,
                       **kwargs
                      ):
        if optimizer.__name__=='SGD':
            optimizer = partial(optimizer,
                momentum=momentum,
                nesterov=kwargs.get('nesterov', True)
            )
        else:
            optimizer = partial(optimizer,
                momentum=momentum,
            )
        return optimizer
    
    
    def _prepare_dl(self, bs=8, shuffle=True):
        dl = ds.get_tf_dataset(shuffle=shuffle)
        dl = dl.map(self.rescale, Learner._AUTOTUNE)
        return dl.batch(bs).prefetch(Learner._AUTOTUNE)

    
    def cyclic_fit(self,
                   epochs,
                   batch_size,
                   lr_range=(1e-4, 1e-2),
                   optimizer=tf.keras.optimizers.SGD,
                   momentum=0.9,
                   validation_data=None,
                   callbacks=None,
                   *args,
                   **kwargs
                  ):
        
        self.max_lr, self.min_lr = lr_range
        
        step_size = 2 * len(self.ds)//batch_size
        
        lr_schedule = tfa.optimizers.Triangular2CyclicalLearningRate(
                                    initial_learning_rate=lr_range[0],
                                    maximal_learning_rate=lr_range[1],
                                    step_size=kwargs.get('step_size', step_size),
                                    scale_mode=kwargs.get('scale', 'cycle'))
        
        
        optimizer = self._get_optimizer(optimizer, momentum=momentum)
        optimizer = optimizer(learning_rate=lr_schedule)
        self.model.optimizer = optimizer
        
        return self.model.fit(
            self._prepare_dl(batch_size, kwargs.get('shuffle', True)),
            validation_data=validation_data,
            epochs=epochs,
            callbacks=callbacks
        )

In [ ]:
from glob import glob
def load_files(path):
    return glob(f'{path}/*/images/*')

def get_label(path):
    return path.split('/')[-3]


ds = Dataset('/data/aniket/tiny-imagenet/data/tiny-imagenet-200/train', image_size=(64, 64))
ds.update_component('get_filenames', load_files)
ds.update_component('get_label', get_label)

get_filenames updated with <function load_files at 0x7f4eae8b9560>
get_label updated with <function get_label at 0x7f4eae8b9dd0>


In [ ]:
learner = Learner(ds=ds, base_model_fn=tf.keras.applications.xception.Xception, name='myModel')

In [ ]:
learner.compile(loss=tf.keras.losses.SparseCategoricalCrossentropy())

In [ ]:
learner.cyclic_fit(5, 64, (1e-4, 1e-2))

Epoch 1/5
    108/Unknown - 95s 879ms/step - loss: 10.9953